In [1]:
from langchain import PromptTemplate, LLMChain
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_community.chat_models import ChatOllama
from langchain_core.output_parsers import PydanticOutputParser
from typing import Optional

title_descr = "The title of the song or album mentioned in the text. If multiple, separate by semicolon. "
performer_descr = "Performer(s) in the text which are associated with the song or album  mentioned. If multiple, separate by semicolon. "

class WorkOfArtSimple(BaseModel):
    title: str = Field(description=title_descr, default=None)
    performer: str = Field(description=performer_descr, default=None)

title_cue_descr = "Text from the source indicating the song title/album title. For instance 'The song' for song 'X' in the phrase 'The song X'."
performer2_descr = "Performers who are not explicitly mentioned in the source text but are relevant."
performer_cue_descr = "Text from the source indicating the performer. For instance 'by' in for performer 'Foo' the phrase 'Song X by Foo'"

class WorkOfArtRich(BaseModel):
    title: str = Field(description=title_descr, default=None)
    title_cue: str = Field(description=title_cue_descr, default=None)
    performer: str = Field(description=performer_descr, default=None)
    performer_unmentioned: str = Field(description=performer2_descr, default=None)
    performer_cue: str = Field(description=performer_cue_descr, default=None)

output_format = WorkOfArtSimple

parser = PydanticOutputParser(pydantic_object=output_format)

model = ChatOllama(model="llama3", temperature=0)


# ZeroShot

In [169]:
template = """Extract the following music attributes from the given Reddit post.
    
    Here are the formatting instructions: 
    {format_instructions} Please only reply with the corresponding output without additional textual explanations.
    If you do not find a corresponding element in the text, assign an empty string to the key.

    Here is the source text:
    '{source_text}'"""

prompt = PromptTemplate(
    template=template,
    input_variables=["source_text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser


In [173]:
import sys
sys.path.append("..")
from tqdm import tqdm
from src.Utils import read_IOB_file, transform_to_dict
from langchain_core.tools import tool

# load data
dataset_id = str(1)
data_path = f"../baseline/music-ner-eacl2023/data/dataset{dataset_id}/test.bio"
texts, labels = read_IOB_file(data_path)

outputs = []
for tokens, iob in tqdm(zip(texts, labels)):

    text = ' '.join(tokens)

    try:
        result = chain.invoke({"source_text": text})
    except Exception as e:
        # If parsing fails, return a WorkOfArtSimple instance with empty strings
        print(f"Parsing failed: {e}")
        result = output_format()
    
    output = result.dict()

    # format output 
    ents = transform_to_dict(tokens, iob)
    output["performer_true"] = ents.get("Artist") or []
    output["title_true"] = ents.get("WoA") or []
    output["title"] = output["title"].split(";") if len(output["title"]) > 0 else []
    output["performer"] = output["performer"].split(";") if len(output["performer"]) > 0 else []
    output["text"] = text
    outputs.append(output)
    

0it [00:00, ?it/s]

600it [02:45,  3.62it/s]


# FewShot

In [ ]:
from langchain import FewShotPromptTemplate

# Few-shot examples (optional)
examples = [
{
    "source_text": "recommend some songs like shape of you by ed sheeran",
    "title": " shape of you",
    "performer": "ed sheeran",
    "performer_unmentioned": "",
    "title_cue": "songs like",
    "performer_cue": "by"
},
{
    "source_text": "i am looking for similar stuff like maroon 5",
    "title": "",
    "performer": "maroon 5",
    "performer_unmentioned": "",
    "title_cue": "",
    "performer_cue": "like"
},
{
    "source_text": "can you give me some nice medieval folk songs",
    "title": "",
    "performer": "",
    "performer_unmentioned": "",
    "title_cue": "",
    "performer_cue": ""
},
{
    "source_text": "can you give me some nice medieval folk songs",
    "title": "",
    "performer": "",
    "performer_unmentioned": "",
    "title_cue": "",
    "performer_cue": ""
},
{
    "source_text": "music like metallicas nothing else matters",
    "title": "nothing else matters",
    "performer": "metallica",
    "performer_unmentioned": "",
    "title_cue": "metallicas",
    "performer_cue": "like"
},
{
    "source_text": "what is the most similar album to abbey road",
    "title": "abbey road",
    "performer": "",
    "performer_unmentioned": "the beatles",
    "title_cue": "most similar album to",
    "performer_cue": ""
},
]

example_template = """Source text: {source_text}, 
Output keys and values: 
    'title': '{title}',
    'performer': '{performer}',
    'performer_unmentioned': '{performer_unmentioned}',
    'title_indicator': '{title_cue}',
    'performer_indicator': '{performer_cue}'
"""

example_prompt = PromptTemplate(
    input_variables=["source_text", "title", 
                     "performer", "performer_unmentioned",
                     "title_indicator", "performer_indicator"], 
                     template=example_template
)

instruction = """Extract the following music attributes from the given Reddit post.
    
    Please only reply with the corresponding output without additional textual explanations.
    If you do not find a corresponding element in the text, assign an empty string to the key."""
suffix = "Here is the source text: {source_text}"

parser = PydanticOutputParser(pydantic_object=WorkOfArtRich)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=instruction + "\nHere are some examples for key value pairs. You have to put these into dictionaries: ",
    suffix=suffix,
    input_variables=["source_text"],
    #partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser


In [220]:
prompt.pretty_print()


Extract the following music attributes from the given Reddit post.
    
    Please only reply with the corresponding output without additional textual explanations.
    If you do not find a corresponding element in the text, assign an empty string to the key.
Here are some examples: 

Source text: recommend some songs like shape of you by ed sheeran, 
Output keys and values: 'title': ' shape of you',
    'performer': 'ed sheeran',
    'performer_unmentioned': '',
    'title_indicator': 'songs like',
    'performer_indicator': 'by'


Source text: i am looking for similar stuff like maroon 5, 
Output keys and values: 'title': '',
    'performer': 'maroon 5',
    'performer_unmentioned': '',
    'title_indicator': '',
    'performer_indicator': 'like'


Source text: can you give me some nice medieval folk songs, 
Output keys and values: 'title': '',
    'performer': '',
    'performer_unmentioned': '',
    'title_indicator': '',
    'performer_indicator': ''


Source text: can you give me

In [200]:
result = chain.invoke({"source_text": text})


In [222]:
outputs2 = []
for tokens, iob in tqdm(zip(texts, labels)):

    text = ' '.join(tokens)

    try:
        result = chain.invoke({"source_text": text})
    except Exception as e:
        # If parsing fails, return a WorkOfArtSimple instance with empty strings
        print(f"Parsing failed: {e}")
        result = output_format()
    
    output = result.dict()

    # format output 
    ents = transform_to_dict(tokens, iob)
    output["performer_true"] = ents.get("Artist") or []
    output["title_true"] = ents.get("WoA") or []
    output["title"] = output["title"].split(";") if output["title"] else []
    output["performer"] = output["performer"].split(";") if output["performer"] else []
    output["text"] = text
    outputs2.append(output)
    

0it [00:00, ?it/s]

Parsing failed: Invalid json output: 'performer': 'radiohead',
'title': 'kid a',
'performer_unmentioned': 'aphex twin',
'title_indicator': 'something similar | kinda a bit like',
'performer_indicator': ''
Parsing failed: Invalid json output: 'performer': 'radiohead',
'title': 'kid a',
'performer_unmentioned': 'aphex twin',
'title_indicator': 'something similar | kinda a bit like',
'performer_indicator': ''

1it [02:02, 122.58s/it]

2it [02:03, 50.76s/it] 

Parsing failed: Invalid json output: 'performer': 'i fight dragons',
'title': '',
'performer_unmentioned': '',
'title_indicator': 'similar to',
'performer_indicator': ''


...

180it [03:35,  2.01it/s]

Parsing failed: Invalid json output: 'performer': '',
'title': '',
'performer_unmentioned': '',
'title_indicator': 'any heavy metal',
'performer_indicator': ''


181it [03:35,  1.19s/it]

Parsing failed: Invalid json output: 'performer': 'metallica',
'title': '',
'performer_unmentioned': '',
'title_indicator': 'songs',
'performer_indicator': ''


KeyboardInterrupt: 

In [205]:
outputs2

[{'title': [],
  'title_cue': None,
  'performer': [],
  'performer_unmentioned': None,
  'performer_cue': None,
  'performer_true': ['radioheads', 'aphex twin'],
  'title_true': ['kid a'],
  'text': 'i love radioheads kid a something similar | kinda a bit like aphex twin maybe'},
 {'title': [],
  'title_cue': None,
  'performer': [],
  'performer_unmentioned': None,
  'performer_cue': None,
  'performer_true': ['i fight dragons'],
  'title_true': [],
  'text': 'anything similar to i fight dragons'},
 {'title': [],
  'title_cue': None,
  'performer': [],
  'performer_unmentioned': None,
  'performer_cue': None,
  'performer_true': ['ccrs'],
  'title_true': ['travelin band'],
  'text': 'music similar to ccrs travelin band'},
 {'title': [],
  'title_cue': None,
  'performer': [],
  'performer_unmentioned': None,
  'performer_cue': None,
  'performer_true': ['boris'],
  'title_true': ['blackout'],
  'text': 'songs similar to blackout by boris'},
 {'title': [],
  'title_cue': None,
  'perf